## 작업 3: ResNet-20을 사용하여 컨볼루션 신경망 모델 생성 및 교육

이 작업에서는 CIFAR-10 교육 데이터로 ResNet 신경망을 교육하여 이미지를 알려진 10가지 범주로 분류합니다. 코드는 MXNet으로 작성됩니다.

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)및 CIFAR-100은 8천만 개의 작은 레이블이 지정된 이미지 데이터 세트의 하위 집합입니다. 이는 Alex Krizhevsky, Vinod Nair 및 Geoffrey Hinton이 수집했습니다.

![](cifar-10.png)

CIFAR-10 데이터 세트는 10개의 클래스에 60,000개의 32x32 색상 이미지로 구성되어 있으며 클래스당 6000개의 이미지가 있습니다. 교육 이미지 50,000개 및 테스트 이미지 10,000개가 있습니다.

데이터 세트는 5개의 교육 배치와 1개의 테스트 배치로 나뉘며, 각각 10,000개의 이미지로 구성되어 있습니다. 테스트 배치에는 각 클래스에서 무작위로 선택한 1,000개의 이미지가 포함되어 있습니다. 교육 배치에는 나머지 이미지가 임의의 순서로 포함되어 있으며, 일부 교육 배치에는 한 클래스의 이미지가 다른 클래스보다 많을 수 있습니다. 전체적으로, 교육 배치에는 각 클래스의 이미지가 정확히 5000개씩 포함되어 있습니다. 다음은 이미지 데이터 세트로 작업하는 일반적인 방법입니다.

- 분류
- 현지화
- 세분화
- 장면 분류
- [장면 구문분석](http://sceneparsing.csail.mit.edu/) - 이미지를 하늘, 도로, 사람, 침대와 같은 의미 체계 범주와 관련된 다양한 이미지 영역으로 분할하고 구문 분석

이미지에 대한 딥 러닝에 대해 자세히 알아보고 싶다면 다음의 강의를 참조하십시오. [CS231n: 시각적 인식을 위한 컨볼루션 신경망](http://cs231n.stanford.edu/slides/2016/winter1516_lecture8.pdf)


**Shift+Enter** 키를 눌러 셀을 실행합니다. 셀 실행이 완료되면 셀 좌측의 숫자가 **In [*]:** 에서 **In [1]** 로 변경됩니다..

In [ ]:
import os, sys
import argparse
import logging
import mxnet as mx
import random
from mxnet.io import DataBatch, DataIter
import numpy as np
import time
import subprocess
import errno
import sagemaker

In [ ]:
!pip install gluoncv

In [ ]:
from __future__ import division

import argparse, time, logging, random, math

import numpy as np
import mxnet as mx

from mxnet import gluon, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms

from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, TrainingHistory

In [ ]:
# Number of GPUs to use
num_gpus = 1
ctx = [mx.gpu(i) for i in range(num_gpus)]

In [ ]:
transform_train = transforms.Compose([
    # Randomly flip the image horizontally
    transforms.RandomFlipLeftRight(),
    # Randomly jitter the brightness, contrast, and saturation of the image
    transforms.RandomColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    # Randomly add noise to the image
    transforms.RandomLighting(0.1),
    # Transpose the image from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    transforms.ToTensor(),
    # Normalize the image with mean and standard deviation calculated across all images
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

In [ ]:
transform_test = transforms.Compose([
    # Transpose the image from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    transforms.ToTensor(),
    # Normalize the image with mean and standard deviation calculated across all images
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

In [ ]:
# Batch size for each GPU
per_device_batch_size = 128
# Number of data loader workers
num_workers = 8
# Calculate effective total batch size
batch_size = per_device_batch_size * num_gpus

# Set train=True for training data
# Set shuffle=True to shuffle the training data
train_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10(train=True).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, last_batch='discard', num_workers=num_workers)

# Set train=False for validation data
val_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10(train=False).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
# Get the model CIFAR_ResNet20_v1, with 10 output classes, without pretrained weights
net = get_model('cifar_resnet20_v1', classes=10, pretrained=False)
net.initialize(mx.init.Xavier(), ctx = ctx)

In [ ]:
# Using stochastic gradient descent
optimizer = 'sgd'

# Set parameters
optimizer_params = {'learning_rate': 0.01, 'wd': 0.0001, 'momentum': 0.9}

# Define the trainer for net
trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)

In [ ]:
# Softmaxcrossentropy loss function
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

# Use accuracy as the training metric
train_metric = mx.metric.Accuracy()
train_history = TrainingHistory(['training-acc', 'validation-acc'])

In [ ]:
def test(net, ctx, val_data):
    '''
    The test function to be used in the training data to check accuracy of the unseen data
    Params:
        ctx: Context describes the device type and ID on which computation should be carried out
        val_data: Validation data to check the accuracy of unseen data
    Returns:
        metrics: Metric name and accuracy
    '''
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)
    return metric.get()

In [ ]:
epochs = 10
lr_decay_count = 0

for epoch in range(epochs):
    tic = time.time()
    train_metric.reset()
    train_loss = 0

    # Loop through each batch of training data
    for i, batch in enumerate(train_data):
        # Extract data and label
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)

        # AutoGrad
        with ag.record():
            output = [net(X) for X in data]
            loss = [loss_fn(yhat, y) for yhat, y in zip(output, label)]

        # Backpropagation
        for l in loss:
            l.backward()

        # Optimize
        trainer.step(batch_size)

        # Update metrics
        train_loss += sum([l.sum().asscalar() for l in loss])
        train_metric.update(label, output)

    name, acc = train_metric.get()
    # Evaluate on validation data
    name, val_acc = test(net, ctx, val_data)

    # Update history and print metrics
    train_history.update([acc, val_acc])
    print('[Epoch %d] train=%f val=%f loss=%f time: %f' %
        (epoch, acc, val_acc, train_loss, time.time()-tic))

# Plot the metric scores
train_history.plot()

이제 10개의 에포크를 실행한 후 80%에 가까운 검증 정확도를 확보해야 합니다. 기존에 존재하는 다른 모델과 비교했을 때 이 모델의 차이점은 무엇입니까?

# 작업 4: 다른 ResNet 모델과 비교

이 작업에서는 `pretrained = True` 및 `pretrained = False` 플래그를 사용하여 서로 다른 두 모델 ResNet-20과 ResNet-56의 네 가지 검증 정확도를 비교합니다. 기계 학습 프로젝트에서는 보통 정확도, 정밀도 또는 재현성과 같은 지표를 사용하여 여러 모델을 비교합니다. 여기에서는 검증 데이터에만 정확도 지표를 사용하십시오.

시작하려면 이전 작업의 코드를 사용하되, 여러 모델을 사용하여 호출할 수 있도록 `model_training_job()` 함수에서 코드를 래핑합니다.

In [ ]:
num_gpus = 1
ctx = [mx.gpu(i) for i in range(num_gpus)]

def model_training_job(model, epochs=10):
    '''
    The function describes the model training job with the specified model using the variable "model".
    The function includes ingesting the data, creating the transforms, and defining the hyperparams
    before you start your training loop.
    Params:
        model: initialized machine learning algorithm you are training
        epochs: number of epochs to train the algorithm; default is 10
    Returns:
        training_history: history of metrics per epoch
    '''
    num_epochs = epochs
    
    transform_train = transforms.Compose([
    # Randomly flip the image horizontally
    transforms.RandomFlipLeftRight(),
    # Randomly jitter the brightness, contrast, and saturation of the image
    transforms.RandomColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    # Randomly add noise to the image
    transforms.RandomLighting(0.1),
    # Transpose the image from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    transforms.ToTensor(),
    # Normalize the image with mean and standard deviation calculated across all images
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
    
    transform_test = transforms.Compose([
    #transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
    
    # Batch size for each GPU
    per_device_batch_size = 128
    # Number of data loader workers
    num_workers = 8
    # Calculate effective total batch size
    batch_size = per_device_batch_size * num_gpus

    # Set train=True for training data
    # Set shuffle=True to shuffle the training data
    train_data = gluon.data.DataLoader(
        gluon.data.vision.CIFAR10(train=True).transform_first(transform_train),
        batch_size=batch_size, shuffle=True, last_batch='discard', num_workers=num_workers)

    # Set train=False for validation data
    val_data = gluon.data.DataLoader(
        gluon.data.vision.CIFAR10(train=False).transform_first(transform_test),
        batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    # Learning rate decay factor
    lr_decay = 0.0001
    # Epochs where learning rate decays
    lr_decay_epoch = [80, 160, np.inf]
    lr_decay_count = 0

    # Using stochastic gradient descent
    optimizer = 'sgd'
    # Set parameters
    optimizer_params = {'learning_rate': 0.01, 'wd': 0.0001, 'momentum': 0.9}

    # Define the trainer for net
    trainer = gluon.Trainer(model.collect_params(), optimizer, optimizer_params)
    
    # Define the loss function
    loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()
    
    # Define the training metric "accuracy" using mx.metric.Accuracy()
    train_metric = mx.metric.Accuracy()
    train_history = TrainingHistory(['training-acc', 'validation-acc'])
    
    print("Starting Training")
    for epoch in range(epochs):
        tic = time.time()
        train_metric.reset()
        train_loss = 0

        # Loop through each batch of training data
        for i, batch in enumerate(train_data):
            #print(f'Epoch: {epoch} Batch: {i}')
            # Extract data and label
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)

            # AutoGrad
            with ag.record():
                output = [model(X) for X in data]
                loss = [loss_fn(yhat, y) for yhat, y in zip(output, label)]

            # Backpropagation
            for l in loss:
                l.backward()

            # Optimize
            trainer.step(batch_size)

            # Update metrics
            train_loss += sum([l.sum().asscalar() for l in loss])
            train_metric.update(label, output)

        name, acc = train_metric.get()
        # Evaluate on Validation data
        name, val_acc = test(model,ctx, val_data)

        # Update history and print metrics
        train_history.update([acc, val_acc])
        print('[Epoch %d] train=%f val=%f loss=%f time: %f' %
            (epoch, acc, val_acc, train_loss, time.time()-tic))

    # Plot the metric scores
    train_history.plot()
    return train_history.history

In [ ]:
trains = {}

print('Training cifar_resnet20_v2 without pretrain')
net_20_f = get_model('cifar_resnet20_v2', classes=10, pretrained=False, ctx=ctx)
net_20_f.initialize(mx.init.Xavier(), ctx = ctx)
trains['cifar_resnet20_v2_f'] = model_training_job(net_20_f,3)

print('Training cifar_resnet56_v2 without pretrain')
net_56_f = get_model('cifar_resnet56_v2', classes=10, pretrained=False, ctx=ctx)
net_56_f.initialize(mx.init.Xavier(), ctx = ctx)
trains['cifar_resnet56_v2_f'] = model_training_job(net_56_f,3)

print('Training cifar_resnet20_v2 with pretrain')
net_20_t = get_model('cifar_resnet20_v2', classes=10, pretrained=True, ctx=ctx)
#net_20_t.initialize(mx.init.Xavier(), ctx = ctx)
trains['cifar_resnet20_v2_t'] = model_training_job(net_20_t,3)

print('Training cifar_resnet56_v2 with pretrain')
net_56_t = get_model('cifar_resnet56_v2', classes=10, pretrained=True, ctx=ctx)
#net_56_t.initialize(mx.init.Xavier(), ctx = ctx)
trains['cifar_resnet56_v2_t'] = model_training_job(net_56_t,3)

알고리즘을 비교하려면 `bokeh` 라이브러리를 사용하여 서로 다른 검증 곡선을 그래프로 표시하여 이들 사이의 정확도를 확인합니다.

In [ ]:
import bokeh
from bokeh.plotting import figure, output_file, show,output_notebook
output_notebook()
def model_comparison(data_type):
    p = figure(plot_width=800, 
               plot_height=400,
               x_axis_label='Number of epochs',
               y_axis_label=f'{data_type} Accuracy',
               toolbar_location='above')
    x = list(range(len(trains['cifar_resnet20_v2_f']['training-acc'])))
    colors = ['green', 'orange', 'blue','red']
    color = colors[:len(trains.keys())]

    for keys,col in zip(trains.keys(),colors):
        print(keys,col)
        acc = trains[keys][f'{data_type}-acc']
        p.line(x,acc, line_width=2,legend=keys,color=col)
        p.circle(x,acc, line_width=2,color=col)
        #show(p)

    p.legend.location = 'bottom_right'
    p.xaxis[0].ticker.desired_num_ticks = len(x)
    show(p)    

model_comparison('validation')

교육 데이터도 살펴보십시오.

In [ ]:
model_comparison('training')

플롯에서 `cifar_resnet20_v2_f`와`cifar_resnet56_v2_f`는 서로 매우 근접하지만 `cifar_resnet20_v2_t` 및`cifar_resnet56_v2_t`에 근접하지 않습니다. 한 가지 차이점은 다른 두 모델보다 훨씬 높은 정확도를 제공하는 모델에 `pretrained = True` 플래그를 추가했다는 점입니다. 

### 질문: 모델이 `pretrained = True` 플래그에 대해 더 높은 정확도를 제공하는 이유는 무엇입니까? 모델이 `pretrained = True` 플래그에 대해 더 높은 정확도를 제공하는 이유는 무엇입니까? 

**답변**: 사전 교육된 컨볼루션 신경망(CNN)모델은 더 큰 데이터 세트에 대해 교육되었으며 때로는 더 긴 시간(더 많은 에포크)동안 실행되는 CNN 모델입니다. 이 실습의 모델은 CIFAR-10 데이터 세트에 대한 교육이었고, 모델이 학습한 초기 가중치는 inital weights(inital weights)에 추가되었습니다. 따라서 처음부터 학습하는 대신 사전 교육된 모델이 학습한 기능과 가중치를 사용하여 교육을 시작합니다. 이를 *증분 교육* 이라고도 합니다. 

대부분의 경우 문제는 이러한 데이터 세트 중 하나와 같은 문제가 아닐 수 있습니다. 예를 들어, 예측하려는 클래스가 CIFAR-10 데이터 세트에 없는 경우 어떻게 해야 합니까? 이러한 경우 처음 몇 개의 계층을 교육할 필요 없이 ImageNet과 같은 관련 대규모 문제의 사전 교육된 모델을 다른 시각적 인식 작업에 사용할 수 있습니다. 이렇게 하면 문제의 이미지를 인식하도록 모델을 교육하는 동안 첫 번째 계층 가중치는 고정되거나 변경되지 않습니다. 이를 *미세 조정* 이라고 합니다.

이 경우 문제의 이미지를 인식하도록 모델을 교육하는 동안 첫 번째 계층 가중치는 고정되거나 변경되지 않습니다. 다른 문제에서 다른 문제로의 지식 전달을 *전송 학습* 이라고 합니다. 다른 문제에서 훈련된 CNN 모델을 사용하고 있기 때문입니다. 이는 일반적으로 학습 속도를 높이고 대규모 교육 데이터 세트에 대한 필요성을 줄이기 위해 수행됩니다.

## 작업 5: Amazon SageMaker 내장 알고리즘을 사용하여 점진적으로 모델을 교육

이제 훈련한 모델을 가져와서 Amazon SageMaker 이미지 분류 알고리즘을 사용하여 모델을 교육합니다. 이 알고리즘은 다중 레이블 분류를 지원하는 지도 학습 알고리즘입니다. 이미지를 입력으로 가져와 해당 이미지에 할당된 하나 이상의 레이블을 출력합니다. 이 알고리즘은 처음부터 훈련하거나 대량의 교육 이미지를 사용할 수 없는 경우 전송 학습을 사용하여 훈련할 수 있는 CNN(ResNet)을 사용합니다.

먼저 생성된 모델의 파라미터를 저장합니다.

In [ ]:
net.save_parameters('cifar10_resnet20_v2_f.params')
#net.summary

경우에 따라 모델 파라미터와 모델 아키텍처를 저장해야 할 수 있습니다. 네트워크가 하이브리드인 경우 네트워크 아키텍처를 파일에 저장할 수도 있으며, 네트워크를 로드하는 데 Python 파일의 네트워크 정의가 필요하지 않습니다.

In [ ]:
net.hybridize()
model_training_job(net, 5)
net.export('cifar10_resnet20_v2_f')

이제 Amazon SageMaker 교육 작업에 사용할 새로운 Amazon Simple Storage Service(Amazon S3)버킷을 생성합니다. `<enter-your-bucket-name>`을 **고유한** 버킷 이름으로 바꿉니다.

In [ ]:
import boto3
import sagemaker
import re
from sagemaker import get_execution_role
import logging
from sagemaker.amazon.amazon_estimator import get_image_uri
from botocore.exceptions import ClientError

sess = sagemaker.Session()

role = get_execution_role()
print(role)

role = get_execution_role()

bucket = '<enter-your-bucket-name>' # Customize to your bucket

s3 = boto3.client('s3')
region = s3.meta.region_name
print(region)
# us-east-1 is s3 default region
try:
    if region == 'us-east-1':
        s3_client = boto3.client('s3')
        s3_client.create_bucket(Bucket=bucket)
        print("Bucket Created")
    else:
        s3_client = boto3.client('s3', region_name=region)
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket, CreateBucketConfiguration=location)
        print("Bucket Created")
except ClientError as e:
    logging.error(e)

# Get the right container image for image training
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')
print(training_image)

이제 버킷이 생성되었으므로 Amazon SageMaker 교육 작업이 액세스할 수 있도록 교육 및 검증 데이터를 Amazon S3 버킷에 업로드합니다. 

Amazon SageMaker 이미지 분류 알고리즘은 파일 모드에서 교육할 수 있도록 **RecordIO** (`application/x-recordio`) 및 **image** (`image/png`, `image/jpeg`, `application/x-image`) 콘텐츠 유형을 모두 지원하며, 파이프 모드에서 교육할 수 있도록 **RecordIO** (`application/x-recordio`) 콘텐츠 유형을 지원합니다. 그러나 증강 매니페스트 형식을 사용하여 RecordIO 파일을 생성하지 않고도 이미지 파일(`image/png`, `image/jpeg`, `application/x-image`)을 사용하여 파이프 모드에서 훈련할 수도 있습니다. 이 알고리즘은 추론에 `image/png`, `image/jpeg` 및 `application/x-image`를 지원합니다. 

본 실습에서는 이 노트북 인스턴스에 이미 제공된 RecordIO 형식을 사용합니다.

In [ ]:
import os 
import urllib.request
import boto3

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

# caltech-256
s3_key = "image-classification-full-training"
s3_train_key = s3_key + "/train"
s3_validation_key = s3_key + "/validation"
s3_train = 's3://{}/{}/'.format(bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(bucket, s3_validation_key)

#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')
upload_to_s3(s3_train_key, 'cifar10_train.rec')
#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')
upload_to_s3(s3_validation_key, 'cifar10_val.rec')

### 교육 작업

이제 필요한 모든 설정을 마쳤으므로 객체 감지기를 교육할 준비가 되었습니다. 시작하려면 sageMaker.estimator.Estimator 객체를 생성합니다. 이 예측기는 교육 작업을 시작합니다.

훈련을 위해 두 가지 종류의 파라미터를 설정해야 합니다. 첫 번째는 훈련 작업에 대한 파라미터입니다. 여기에는 다음이 포함됩니다.
- **교육 인스턴스 수**: 교육을 실행할 인스턴스 수. 인스턴스 수가 1보다 크면 이미지 분류 알고리즘이 분산 설정에서 실행됩니다.
- **교육 인스턴스 유형**: 교육을 실행할 시스템의 유형입니다. 일반적으로 이 교육에서는 GPU 인스턴스가 사용됩니다.
- **출력 경로**: 교육 출력이 저장되는 Amazon S3 폴더입니다.

Amazon SageMaker CreateTrainingJob API를 사용하여 교육을 실행합니다.

In [ ]:
import sagemaker

s3_output_location = 's3://{}/{}/output'.format(bucket, s3_key)
cifar = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p2.xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

알고리즘에 특화된 하이퍼파라미터는 다음과 같습니다.
- **num_layers**: 네트워크의 계층 수(깊이). 이 샘플은 18을 사용하지만 50 및 152와 같은 다른 값을 사용할 수 있습니다.
- **image_shape**: 네트워크의 입력 이미지 크기, `num_channels, height, width`. 실제 이미지 크기보다 크지 않아야 합니다. 실제 이미지 크기보다 크지 않아야 합니다.
- **num_classes**: 새 데이터 세트에 대한 출력 클래스 수. ImageNet은 1,000개의 출력 클래스를 사용하여 교육되었지만 미세 조정을 위해 출력 클래스 수를 변경할 수 있습니다. Caltech 256 데이터 세트의 경우 256개의 객체 범주와 1개의 clutter class를 포함하고 있으므로 257이 사용됩니다. 
- **num_training_samples**: 총 교육 샘플 수. 현재 분할된 Caltech 256 데이터 세트에 대해 15,240로 설정됩니다.
- **mini_batch_size**: 각 미니 배치에 사용되는 교육 샘플 수. 분산 교육에서 배치당 사용되는 교육 샘플 수는 N * mini_batch_size이며, 여기서 N은 훈련이 실행되는 호스트 수입니다.
- **epochs**: 총 교육 에포크 수
- **learning_rate**: 교육 학습률
- **top_k**: 교육 중 top-k 정확도 보고

In [ ]:
cifar.set_hyperparameters(num_layers=20, 
                             image_shape = "3,32,32",
                             num_classes=10,
                             num_training_samples=50000,
                             mini_batch_size=128,
                             epochs=10,
                             learning_rate=0.1,
                             top_k=2)

Amazon SageMaker 교육 작업에서 사용하는 입력 데이터에 대한 정의를 생성합니다.

In [ ]:
train_data = sagemaker.session.s3_input(s3_train, distribution='FullyReplicated', 
                            content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation, distribution='FullyReplicated', 
                            content_type='application/x-recordio', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
cifar.fit(inputs=data_channels, logs=True)

## 작업 6: Amazon SageMaker 엔드포인트를 사용하여 추론을 위한 모델 준비

이제 교육된 모델을 사용하여 추론을 수행할 수 있습니다. 이 예제에서는 CIFAR-10 데이터 세트에서 10개의 클래스를 예측합니다. 예측기에서 배포 메서드를 사용하여 생성된 모델을 배포할 수 있습니다. 그러면 새 Amazon SageMaker 엔드포인트가 생성됩니다. 다음 두 가지 방법 중 하나로 예측을 가져오도록 배포할 수 있습니다.
- 한 번에 하나의 예측을 가져오도록 영구 엔드포인트를 설정하려면 Amazon SageMaker 호스팅 서비스를 사용하십시오.
- 전체 데이터 세트에 대한 예측을 가져오려면 Amazon SageMaker 배치 변환을 사용하십시오.

이 작업에서는 Amazon SageMaker 호스팅 서비스를 사용하여 호출당 단일 예측을 가져오도록 영구 엔드포인트를 설정합니다.

Amazon SageMaker 호스팅 서비스를 사용한 모델 배포는 세 단계로 이루어집니다.

1. **Amazon SageMaker에서 모델 생성**: 모델을 생성하여 Amazon SageMaker에서 모델 구성 요소를 찾을 수 있는 위치를 알려줍니다. 여기에는 모델 결과물이 저장되는 Amazon S3 경로와 추론 코드가 포함된 이미지의 도커 레지스트리 경로가 포함됩니다. 후속 배포 단계에서는 이름을 기준으로 모델을 지정합니다.

2. **HTTPS 엔드포인트에 대한 엔드포인트 구성 생성**: Amazon SageMaker가 각 프로덕션 변형을 호스팅하기 위해 시작하려는 ML 컴퓨팅 인스턴스와 프로덕션 변형에 있는 하나 이상의 모델 이름을 지정합니다.

3. **HTTPS 엔드포인트 생성**: Amazon SageMaker에 엔드포인트 구성을 제공합니다. 이 서비스는 ML 컴퓨팅 인스턴스를 시작하고 구성에 지정된 대로 모델을 배포합니다. 자세한 내용은 CreateEndpoint API를 참조하십시오. 클라이언트 애플리케이션은 모델로부터 추론을 얻기 위해 Amazon SageMaker 런타임 HTTPS 엔드포인트로 요청을 전송합니다. API에 대한 자세한 내용은 InvokeEndpoint API를 참조하십시오.

In [ ]:
cifar_classifier = cifar.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

현재 배포된 엔드포인트가 있는 경우 다음 명령을 사용하여 엔드포인트를 업데이트할 수 있습니다. 이렇게 하려면 명령의 주석 처리를 해제하고 `<endpoint_name>`을 현재 실행 중인 엔드포인트의 이름으로 바꿉니다. 엔드포인트가 업데이트될 때까지 기다린 후 다음 코드 셀을 실행합니다. 

In [ ]:
#cifar_classifier = cifar.deploy(endpoint_name = <endpoint_name>, 
#                                update_endpoint=True, 
#                                initial_instance_count = 1, 
#                                instance_type = 'ml.m4.xlarge')

엔드포인트가 업데이트 또는 생성되었는지 확인하려면 ** boto3** 를 `DescribeEndpoint`에 사용합니다. 상태가 **InService** 로 변경될 때까지 계속 진행하지 마십시오.

In [ ]:
sm = boto3.client('sagemaker')

In [ ]:
describe_endpoint = sm.describe_endpoint(EndpointName=cifar_classifier.endpoint)
print(f"The status of the endpoint is {describe_endpoint['EndpointStatus']} ")
describe_endpoint

이제 변수 `val_data`에 검증 데이터가 이미 있습니다. 예측에 사용할 Gluon에서 원시 이미지 데이터 `img_data`를 가져옵니다. 이 엔드포인트를 예측하는 두 가지 방법으로 사용합니다.
1. 모델을 개발할 때 원시 Gluon CIFAR-10 검증 이미지 데이터 `img_data`로 예측합니다.
2. 앱에 모델을 배포할 때 URL로 예측하면 앱이 이미지 URL을 가지게 됩니다.

### 원시 Gluon CIFAR-10 검증 이미지 데이터로 예측

In [ ]:
from mxnet import autograd, gluon, image, init, nd
from matplotlib.pylab import imshow

img_data = gluon.data.vision.CIFAR10(train=False)

label_dict = {0:"airplane", 1:"automobile", 2:"bird", 3:"cat", 4:"deer",
              5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"
             }

Amazon SageMaker 엔드포인트는 한 번에 하나의 이미지를 예측합니다. 첫 번째 이미지를 선택하십시오.

In [ ]:
sample = img_data[0]
data = sample[0]
label = sample[1]

imshow(data.asnumpy())

이미지를 예측합니다.

In [ ]:
import cv2
import json

cifar_classifier.content_type = 'application/x-image'

sample_imgs, sample_labels = img_data[:10]

for img, label in zip(sample_imgs, sample_labels): 
    payload = cv2.imencode('.jpeg', img.asnumpy())[1].tobytes()

    predictor_prob = cifar_classifier.predict(payload)
    pred = np.argmax(json.loads(predictor_prob))
    print(f"Prediction: {pred}-{label_dict[pred]}, True Label: {label}-{label_dict[label]} " )

### URL로 예측

애플리케이션 개발을 위해서라면 `model.predict` 는 완벽하게 작동합니다. 하지만 모델을 애플리케이션에 배포할 때는 `sagemaker-runtime` 라이브러리를 사용해서 `invoke_endpoint` API를 호출하여 예측을 가져와야 합니다.

**참고** `response ['Body'].read()`는 `invoke_endpoint` API를 호출할 때마다 한 번만 호출할 수 있습니다.

In [ ]:
import requests
from IPython.display import Image

urls = 'https://cdn.pixabay.com/photo/2013/06/08/04/17/ferry-boat-123059__340.jpg'

display(Image(requests.get(urls).content))


payload = requests.get(urls).content

ENDPOINT_NAME = cifar_classifier.endpoint
runtime= boto3.client('runtime.sagemaker')
response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/x-image',
                                       Body=payload)

pred = np.argmax(json.loads(response['Body'].read()))
label_dict[pred]

## 작업 7: 증분 교육 준비(선택 사항)

이제 이전 작업에서 생성된 모델을 사용하여 동일한 데이터 세트로 다른 교육을 시작합니다. 이 새로운 교육은 이전 작업에서 생성된 모델을 사용하므로 더 높은 정확도로 시작됩니다.

In [ ]:
# Print the location of the model data from the previous tasks
print(cifar.model_data)

# Prepare model channel in addition to train and validation
model_data = sagemaker.session.s3_input(cifar.model_data, distribution='FullyReplicated', 
                              s3_data_type='S3Prefix', content_type='application/x-sagemaker-model')

data_channels = {'train': train_data, 'validation': validation_data, 'model': model_data}

In [ ]:
cifar.fit(inputs=data_channels, logs=True)

## 실습 완료

축하합니다! 본 실습을 완료했습니다. 실습 환경을 정리하려면 다음을 수행하십시오.

- 노트북 파일을 닫습니다.
- **Quit** 을 클릭하여 Jupyter 노트북에서 로그아웃한 다음 탭을 닫습니다.
- AWS Management Console에서 로그 아웃하려면 콘솔 맨 위에 있는 **awsstudent**을 클릭한 후 **Sign Out**을 클릭합니다.
- **End Lab** 을 클릭하여 Qwiklabs에서 실습 세션을 종료합니다.